In [ ]:
%matplotlib inline

import numpy as np # NUMPY
import pandas as pd # PANDAS
import matplotlib.pyplot as plt # MATPLOTLIB
import cv2
import math
import os

In [ ]:
'''
!git clone https://github.com/aliciaamaliaalvarezmon/SynthesisEmotions.git
%run /content/SynthesisEmotions/Notebook/auxfunctions.ipynb
%run /content/SynthesisEmotions/Notebook/ExpressionMapFunctionsAux.ipynb
%run /content/SynthesisEmotions/Notebook/SOM2ver.ipynb
#import subprocess
#!chmod a+x '/content/SynthesisEmotions/cartoonTexture'
'''


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pickle as pk

In [ ]:

def doPCA(data, comp):
    std_scaler = StandardScaler()
    scaler = std_scaler.fit(data)
    scaled_landDiff = scaler.transform(data)
    pca = PCA(n_components=comp)
    #pca.fit_transform(scaled_landDiff)
    b_shape = pca.fit_transform(scaled_landDiff)
    print(sum(pca.explained_variance_ratio_))
    print(b_shape.shape)
    return b_shape, pca, scaler

def transformPCA(data, pca, scaler):
    scaled_landDiff = scaler.transform(data)
    return pca.transform(scaled_landDiff)

In [ ]:
##Vamos a necesitar esto para los V1,V2,V3 anticuados. Usarlo y luego dejarlo comentado
def discard_all_extra_neutrals(data, path_folder):
  data = np.asarray(data)
  image_names = np.sort(os.listdir(path_folder))
  current_neutral_index = -1
  my_data_for_PCA_SNeu = []
  current_subject = 'start'

  for i in range(0, len(images_names)):
    if int(images_names[i].split('_')[5].split('.')[0]) == 1:
      if current_subject != images_names[i].split('_')[3]:
        current_subject = images_names[i].split('_')[3]
        my_data_for_PCA_SNeu.append(data[i])
    else:
        my_data_for_PCA_SNeu.append(data[i])

  my_data_for_PCA_SNeu = np.asarray(my_data_for_PCA_SNeu)
  return my_data_for_PCA_SNeu


In [ ]:
def get_first_frames_and_images(path,my_data):
    first_images = []
    first_frames = []
    total = 0

    current_neutral_index = 0
    images_names = np.sort(os.listdir(path))
    for i in range(0, len(images_names)):
        print(images_names[i].split('_'))
        if int(images_names[i].split('_')[5].split('.')[0]) == 1:
            first_frames.append(my_data.iloc[i])
            first_images.append(path +'/'+ images_names[i])

    first_frames = np.asarray( first_frames, dtype=object)
    first_images = np.asarray( first_images, dtype=object)

    return first_frames, first_images

def get_first_frames_and_images_test(path,my_data):
    first_images = []
    first_frames = []
    total = 0

    current_neutral_index = 0
    images_names = np.sort(os.listdir(path))
    for i in range(0, len(images_names)):
      first_frames.append(my_data.iloc[i])
      first_images.append(path +'/'+ images_names[i])

    first_frames = np.asarray( first_frames, dtype=object)
    first_images = np.asarray( first_images, dtype=object)

    return first_frames, first_images

In [ ]:
def align_and_transform_datavector_of_images(reference_image, reference_landmarks, landmarks, images, pathi):
    transformed_landmarks_total = []
    for i in range(0, len(landmarks)):
        tr_Y_img, Z_pts = align_and_transform_image(reference_image, reference_landmarks, landmarks.iloc[i], images[i])
        transformed_landmarks_total.append(Z_pts)
        filename = 'warped_image_procustres_'+ str(images[i].split('/')[8])+'.png'
        path = pathi
        print(os.path.join(path , filename))
        cv2.imwrite(os.path.join(path , filename), tr_Y_img)
    transformed_landmarks_total = np.asarray(transformed_landmarks_total)
    return transformed_landmarks_total

def get_inten_emo(my_data_2):
  Ai = my_data_2.max()
  data_2_intensity = my_data_2
  data_2_intensity_div = data_2_intensity/Ai
  data_2_intensity_sum =  np.sum(data_2_intensity_div, axis=1)
  data_2_intensity_final = data_2_intensity_sum/136
  return data_2_intensity_final

In [ ]:
def fill_struct_no_neu(my_data_2, data_2_intensity_final, structure_vector, no_neutral_im):
    cant_features = 22
    current_sub = 'start'
    ab = 0
    structure_feature_vector = np.zeros((len(my_data_2), cant_features))
    current_features_index = -1
    for i in range(0, len(data_2_intensity_final)):
        print('fill')
        print(no_neutral_im[ab].split('_'))
        if  no_neutral_im[ab].split('_')[3] != current_sub:
            current_sub = no_neutral_im[ab].split('_')[3]
            current_features_index = current_features_index + 1
        vector = structure_vector[:,current_features_index]#columna i
        vector = np.append(vector,data_2_intensity_final.iloc[i])
        structure_feature_vector[ab] = vector
        ab = ab + 1
    return structure_feature_vector

In [ ]:
def cartoon_texture_decomposition(images_path, subject_images, cartoon_folder_path, texture_folder_path,  forShow='True', j=0.2):
    for i in range (0,  len(subject_images)):
        subject = subject_images[i].split('/')[0].split('_')[3]
        serie = subject_images[i].split('/')[0].split('_')[4]
        frame = subject_images[i].split('/')[0].split('_')[5].split('.')[0]
        print(subject,serie, frame)
        image_path = images_path + '/' + subject_images[i]
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img2 = img[40:450, 170:580]
        cv2.imwrite(cropped_img_path, img2)
        cartoon_path = cartoon_folder_path + "/cartoonimg"+'_'+ str(int(j*100))+'_' + subject +'_' +serie +'_'+frame +".png"
        texture_path = texture_folder_path +"/textureimg"+'_'+ str(int(j*100))+'_' + subject +'_' +serie +'_'+frame +".png"
        if(forShow == 'True'):
          subprocess.check_output(['/content/SynthesisEmotions/cartoonTexture',cropped_img_path, str(j), cartoon_path, texture_path])
        else:
          subprocess.check_output(['/content/cartoonTextureNew',cropped_img_path, str(j), cartoon_path, texture_path])
    return 0

In [ ]:
def postcartoon(cartimlist):
  imcart = []
  for i in cartimlist:
    img = cv2.imread(i, cv2.IMREAD_GRAYSCALE)
    img_mean = img/np.mean(img)
    imcart.append(img_mean.flatten())
  return imcart

def diffpostcartoon(cartarray, cartimlist):
  imcartoon = []
  index = -1
  subject = 'start'
  current_image = cartarray[0]
  for i in range(0, len(cartimlist)):
    print(cartimlist[i].split('/'))
    if (int(cartimlist[i].split('/')[3].split('_')[4].split('.')[0]) == 1):
      index = i
      current_image = cartarray[i]
    imcartoon.append(cartarray[i]- current_image)
  return imcartoon

def eliminate_extraneutrals(cartimlist, cartoondiff):
  no_neutral_extra_im = []
  cheking_order = []
  current_sub = 'start'
  for i in range (0, len(cartimlist)):
    if cartimlist[i].split('_')[3] != current_sub:
      current_sub = cartimlist[i].split('_')[3]
      no_neutral_extra_im.append(cartoondiff[i])
      cheking_order.append(cartimlist[i])
    if int(cartimlist[i].split('_')[5].split('.')[0]) != 1:
      no_neutral_extra_im.append(cartoondiff[i])
      cheking_order.append(cartimlist[i])
  no_neutral_extra_im = np.asarray(no_neutral_extra_im)
  no_neutral_extra_im.shape
  return no_neutral_extra_im, cheking_order

def posttexture(teximlist):
  imtext = []
  for i in teximlist:
    img = cv2.imread(i, cv2.IMREAD_GRAYSCALE)
    imtext.append(img.flatten())

  imtextarray = np.asarray(imtext)
  imtextarray.shape
  return imtextarray

def struct_read(path, chunk_size = 100):
  file_path = path
  df_list = []
  for chunk in pd.read_csv(file_path, chunksize=chunk_size, header = None):
    df_list.append(chunk)
  df = pd.concat(df_list)
  return df  #esto equivale a tirar por balde el ahorro. creo


In [ ]:
class EmotionClassifier():
    """
    The 2-D, rectangular grid self-organizing map class using Numpy.
    """
    def __init__(self, training_data_images, training_data_landmarks, aligned_training_images_path, aligned_training_landmarks_path, shape_diff_path,  m=9, n=9, lr=0.25, sigma=1.75, dim = 392):


        # Initialize descriptive features of SOM
        self.m = m
        self.n = n
        self.shape = (m, n)
        self.lr = lr
        self.sigma = sigma
        self.dim = dim



        self.training_set_img_folder = training_data_images
        self.training_set_landmarks_folder = training_data_landmarks

        self.aligned_training_images_path = aligned_training_images_path
        self.aligned_training_landmarks_path = aligned_training_landmarks_path
        self.shape_diff_path = shape_diff_path
        self.aligned_test_images_path = None
        self.aligned_test_landmarks_path = None
        self.shape_diff_path_test = None

        # Initialize weights

        self.ref_img = None
        self.ref_landmarks = None

        self.shape_pca = None
        self.structure_pca = None
        self.cartoon_pca = None
        self.texture_pca = None

        self.shape_scaler = None
        self.structure_scaler = None
        self.cartoon_scaler = None
        self.texture_scaler = None

        self.training_set_features = None
        self.test_set_features = None

        self.som = None



        # Set after fitting


    def _change_training_img_loc(self, loc):
        """
        Return the indices of an m by n array.
        """
        self.training_set_img_folder = loc
        return self.training_set_img_folder

    def _change_test_img_loc(self, loc):
        """
        Return the indices of an m by n array.
        """
        self.test_set_img_folder = loc
        return self.test_set_img_folder

    def _change_training_lan_loc(self, loc):
        """
        Return the indices of an m by n array.
        """
        self.training_set_landmarks_folder = loc
        return self.training_set_landmarks_folder

    def _change_test_lan_loc(self, loc):
        """
        Return the indices of an m by n array.
        """
        self.test_set_landmarks_folder = loc
        return self.test_set_landmarks_folder

    def _save_shape_pca(self):
      pk.dump(self.shape_pca, open("shape_pca.pkl","wb"))

    def _load_shape_pca(self):
      self.shape_pca = pk.load(open("shape_pca.pkl","rb"))
      return self.shape_pca

    def _save_shape_scaler(self):
      pk.dump(self.shape_scaler, open("shape_scaler.pkl","wb"))

    def _load_shape_scaler(self):
      self.shape_scaler = pk.load(open("shape_scaler.pkl","rb"))
      return self.shape_scaler

    def _save_structure_pca(self):
      pk.dump(self.structure_pca, open("structure_pca.pkl","wb"))

    def _load_structure_pca(self):
      self.strucure_pca = pk.load(open("structure_pca.pkl","rb"))
      return self.strucure_pca

    def _save_structure_scaler(self):
      pk.dump(self.structure_scaler, open("structure_scaler.pkl","wb"))

    def _load_structure_scaler(self):
      self.strucure_scaler = pk.load(open("structure_scaler.pkl","rb"))
      return self.strucure_scaler

    def _save_cartoon_pca(self):
      pk.dump(self.cartoon_pca, open("cartoon_pca.pkl","wb"))

    def _load_cartoon_pca(self):
      self.cartoon_pca = pk.load(open("cartoon_pca.pkl","rb"))
      return self.cartoon_pca

    def _save_cartoon_scaler(self):
      pk.dump(self.cartoon_scaler, open("cartoon_scaler.pkl","wb"))

    def _load_cartoon_scaler(self):
      self.cartoon_scaler = pk.load(open("cartoon_scaler.pkl","rb"))
      return self.cartoon_scaler

    def _save_texture_pca(self):
      pk.dump(self.texture_pca, open("texture_pca.pkl","wb"))

    def _load_texture_pca(self):
      self.texture_pca = pk.load(open("texture_pca.pkl","rb"))
      return self.texture_pca

    def _save_texture_scaler(self):
      pk.dump(self.texture_scaler, open("texture_scaler.pkl","wb"))

    def _load_texture_scaler(self):
      self.texture_scaler = pk.load(open("texture_scaler.pkl","rb"))
      return self.texture_scaler


    def _get_training_shape_feature(self, pca_comp = 29):

      images = np.sort(np.asarray(self.training_set_img_folder))
      landmarks = np.sort(np.asarray(self.training_set_landmarks_folder))
      frames_lmarks_XYord = get_landmarks_of_folder(landmarks)
      my_df = pd.DataFrame(frames_lmarks_XYord)
      intercalated_data = intercalate_data_of_datavectors(my_df, 0)
      my_dfc = pd.DataFrame(intercalated_data)

      my_landmarks = my_dfc
      pathi = self.aligned_training_images_path

      self.ref_img = images[0]
      self.ref_landmarks = my_landmarks.iloc[0]
      transformed_landmarks_total = align_and_transform_datavector_of_images(images[0], my_landmarks.iloc[0], my_landmarks, images, pathi)

      transformed_final = transformed_landmarks_total[0].flatten('F')
      for i in range (1, len(transformed_landmarks_total)):
        transformed_final = np.vstack((transformed_final, transformed_landmarks_total[i].flatten('F')))

      my_df = pd.DataFrame(transformed_final)
      my_df.to_csv(self.aligned_training_landmarks_path,header = False, index= False)

      ind = 0
      my_data_aligned_diff = []
      images_names = images
      current_neutral_index = 0
      for i in range(0, len(images_names)):
        if int(images_names[i].split('_')[2].split('.')[0]) == 1:
          current_neutral_index = i
          #print('cambie neutral '+ str(current_neutral_index))
        my_data_aligned_diff.append(my_df.iloc[i]- my_df.iloc[current_neutral_index])
      my_data_aligned_diff = np.asarray(my_data_aligned_diff)

      my_data_forPCA = pd.DataFrame(my_data_aligned_diff)
      my_data_forPCA.to_csv(self.shape_diff_path,header = False, index= False)

      shape,pac, scaler= doPCA(my_data_forPCA, pca_comp)

      self.shape_pca = pac
      self.shape_scaler = scaler
      return shape

    def _get_training_structure_feature(self, structure_features_path, pca_comp = 13):
      my_data = pd.read_csv(self.aligned_training_landmarks_path,header = None)
      subjects_images = self.aligned_training_images_path
      first_frames = get_first_frames(subjects_images,my_data)
      first_images = get_first_images(subjects_images,my_data)
      intercalated_first_frames = change_to_intercalate_order(first_frames)
      structure_vector = do_the_structure_points_and_stack(intercalated_first_frames, first_images)
      my_data_2 = pd.read_csv(self.shape_diff_path,header = None)
      data_2_intensity_final = get_inten_emo(my_data_2)
      images_names = np.sort(os.listdir(self.aligned_training_images_path))

      structure_feature_vector = fill_struct_no_neu(my_data_2, data_2_intensity_final,structure_vector,  images_names)
      structure_feature_vector = np.asarray(structure_feature_vector)
      my_df = pd.DataFrame(structure_feature_vector)
      my_df.to_csv(structure_features_path,header = False, index= False)
      my_data_forPCAss = pd.read_csv(structure_features_path,header = None)
      structure, pac,scaler = doPCA(my_data_forPCAss,pca_comp)

      self.structure_pca = pac
      self.structure_scaler = scaler
      return structure


    def _get_training_cartoon_and_texture_feature(self, cropped_img_path, cartoon_folder_path, texture_folder_path, cartoon_save_path, texture_save_path,make ='True',  forShow='True', cartoon_pca_comp = 100, texture_pca_comp = 250, j =0.2):
      subjects_images = np.sort(os.listdir(self.aligned_training_images_path))
      if(make == 'True'):
        cartoon_texture_decomposition(self.aligned_training_images_path, subjects_images, cartoon_folder_path, texture_folder_path, forShow, j)

      path = cartoon_folder_path
      cartoon_images = []
      for i in np.sort(os.listdir(cartoon_folder_path)):
          cartoon_images.append( path +'/'+ i)
      texture_images = []
      path = texture_folder_path
      for i in np.sort(os.listdir(texture_folder_path)):
          texture_images.append( path + '/' +i)

      imcart1 = postcartoon(cartoon_images)
      imcartoon1 = diffpostcartoon(imcart1, cartoon_images)
      print(len(imcartoon1))

      np.savetxt(cartoon_save_path, imcartoon1, delimiter=",")

      df = struct_read(cartoon_save_path,100)

      cartoon, pac, scaler = doPCA(df, cartoon_pca_comp)
      self.cartoon_pca = pac
      self.cartoon_scaler = scaler

      imtext1 = posttexture(texture_images)

      np.savetxt(texture_save_path, imtext1, delimiter=",")

      df = struct_read(texture_save_path,100)

      texture, pac, scaler = doPCA(df, texture_pca_comp)
      self.texture_pca = pac
      self.texture_scaler = scaler

      return cartoon, texture

    def _get_training_feature_vector(self,structure_features_path,cropped_img_path,cartoon_folder_path,texture_folder_path,cartoon_save_path, texture_save_path, shape_pca_comp = 29, struct_pca_comp = 13, cartoon_pca_comp = 100, texture_pca_comp = 350):
      shape = self._get_training_shape_feature(shape_pca_comp)
      struct = self._get_training_structure_feature(structure_features_path, struct_pca_comp)
      cartoon, texture = self._get_training_cartoon_and_texture_feature(cropped_img_path, cartoon_folder_path, texture_folder_path, cartoon_save_path, texture_save_path, cartoon_pca_comp, texture_pca_comp)
      b_vector = np.concatenate((shape, struct), axis = 1)
      b_vector = np.concatenate((b_vector, cartoon), axis = 1)
      b_vector = np.concatenate((b_vector, texture), axis = 1)
      self.training_set_features = b_vector
      return b_vector


    def _get_testing_shape_feature(self,  testing_set_img_folder,testing_set_landmarks_folder,aligned_testing_images_path, aligned_testing_landmarks_path, shape_test_diff_path):
      #_get_testing_shape_feature(testing_img_set, testing_lab_set, alig_im_test, alig_lan_test, diff_path_test)
      print('llegue')
      print(np.sort(np.asarray(testing_set_img_folder)))
      images = np.sort(np.asarray(testing_set_img_folder))
      landmarks = np.sort(np.asarray(testing_set_landmarks_folder))
      print('llegue')
      frames_lmarks_XYord = get_landmarks_of_folder(landmarks)
      print('llegue 2')
      my_df = pd.DataFrame(frames_lmarks_XYord)
      intercalated_data = intercalate_data_of_datavectors(my_df, 0)
      print('llegue 3')
      my_dfc = pd.DataFrame(intercalated_data)

      my_landmarks = my_dfc
      pathi = aligned_testing_images_path
      self.aligned_test_images_path = aligned_testing_images_path
      # transformed_landmarks_total = align_and_transform_datavector_of_images(images[0], my_landmarks.iloc[0], my_landmarks, images, pathi)

      transformed_landmarks_total = align_and_transform_datavector_of_images(self.ref_img, self.ref_landmarks, my_landmarks, images, pathi)
      print('llegue 4')
      transformed_final = transformed_landmarks_total[0].flatten('F')
      for i in range (1, len(transformed_landmarks_total)):
          transformed_final = np.vstack((transformed_final, transformed_landmarks_total[i].flatten('F')))
      print('llegue 5')
      my_df = pd.DataFrame(transformed_final)
      my_df.to_csv(aligned_testing_landmarks_path,header = False, index= False)
      self.aligned_test_landmarks_path = aligned_testing_landmarks_path

      ind = 0
      my_data_aligned_diff = []
      images_names = images
      current_neutral_index = 0
      for i in range(0, len(images_names)):
        if int(images_names[i].split('_')[2].split('.')[0]) == 1:
          current_neutral_index = i

        my_data_aligned_diff.append(my_df.iloc[i]- my_df.iloc[current_neutral_index])
      my_data_aligned_diff = np.asarray(my_data_aligned_diff)

      my_data_forPCA = pd.DataFrame(my_data_aligned_diff)
      my_data_forPCA.to_csv(shape_test_diff_path,header = False, index= False)
      self.shape_diff_path_test = shape_test_diff_path

      shape = transformPCA(my_data_forPCA, self.shape_pca, self.shape_scaler)

      return shape

    def _get_testing_structure_feature(self, structure_features_path_test):
      my_data = pd.read_csv(self.aligned_test_landmarks_path,header = None)
      subjects_images = self.aligned_test_images_path
      print(self.aligned_test_landmarks_path)
      print(self.aligned_test_images_path)
      first_frames, first_images =get_first_frames_and_images_test(subjects_images,my_data)
      #first_frames = get_first_frames(subjects_images,my_data)
      #first_images = get_first_images(subjects_images,my_data)
      intercalated_first_frames = change_to_intercalate_order(first_frames)
      print(np.asarray(intercalated_first_frames).shape)
      print(np.asarray(first_frames).shape)
      structure_vector = do_the_structure_points_and_stack(intercalated_first_frames, first_images)
      my_data_2 = pd.read_csv(self.shape_diff_path_test,header = None)
      data_2_intensity_final = get_inten_emo(my_data_2)
      images_names = np.sort(os.listdir(self.aligned_test_images_path))
      print(structure_vector.shape)
      structure_feature_vector = fill_struct_no_neu(my_data_2, data_2_intensity_final,structure_vector,  images_names)
      structure_feature_vector = np.asarray(structure_feature_vector)
      my_df = pd.DataFrame(structure_feature_vector)
      my_df.to_csv(structure_features_path_test,header = False, index= False)
      my_data_forPCAss = pd.read_csv(structure_features_path_test,header = None)

      structure = transformPCA(my_data_forPCAss, self.structure_pca, self.structure_scaler)

      return structure


    def _get_testing_cartoon_and_texture_feature(self, cropped_img_path_test, cartoon_folder_path_test, texture_folder_path_test, cartoon_save_path_test, texture_save_path_test, make = 'True'):
      subjects_images = np.sort(os.listdir(self.aligned_test_images_path))
      print(self.aligned_test_images_path)
      if make == 'True':
        cartoon_texture_decomposition(self.aligned_test_images_path, subjects_images, cartoon_folder_path_test, texture_folder_path_test)

      path = cartoon_folder_path_test
      cartoon_images = []
      for i in np.sort(os.listdir(cartoon_folder_path_test)):
        cartoon_images.append( path +'/'+ i)
      texture_images = []
      path = texture_folder_path_test
      for i in np.sort(os.listdir(texture_folder_path_test)):
        texture_images.append( path + '/' +i)
      print(len(cartoon_images))
      imcart1 = postcartoon(cartoon_images)
      print(len(imcart1))
      imcartoon1 = diffpostcartoon(imcart1, cartoon_images)
      print(len(imcartoon1))

      np.savetxt(cartoon_save_path_test, imcartoon1, delimiter=",")

      df = struct_read(cartoon_save_path_test,100)

      cartoon = transformPCA(df, self.cartoon_pca, self.cartoon_scaler)

      imtext1 = posttexture(texture_images)

      np.savetxt(texture_save_path_test, imtext1, delimiter=",")

      df = struct_read(texture_save_path_test,100)

      texture = transformPCA(df, self.texture_pca, self.texture_scaler)

      return cartoon, texture

    def _get_testing_feature_vector(self,aligned_testing_images_path, aligned_testing_landmarks_path, shape_test_diff_path, testing_set_img_folder,testing_set_landmarks_folder,  structure_features_path,cropped_img_path,cartoon_folder_path,texture_folder_path,cartoon_save_path, texture_save_path):
      shape = self._get_testing_shape_feature( aligned_testing_images_path, aligned_testing_landmarks_path, shape_test_diff_path, testing_set_img_folder,testing_set_landmarks_folder)
      struct = self._get_testing_structure_feature(structure_features_path)
      cartoon, texture = self._get_testing_cartoon_and_texture_feature(cropped_img_path, cartoon_folder_path, texture_folder_path, cartoon_save_path, texture_save_path)
      b_vector_test = np.concatenate((shape, struct), axis = 1)
      b_vector_test = np.concatenate((b_vector_test, cartoon), axis = 1)
      b_vector_test = np.concatenate((b_vector_test, texture), axis = 1)
      self.testing_set_features = b_vector_test
      return b_vector_test

    def get_som(self, max_iter = 3000):
      self.som = SOM(self.m, self.n, self.dim, self.lr, self.sigma)
      return self.som

    def fit_som(epo = 1):
      return self.som.fit(training_set_features, epo)

    def get_labelAndTarget(training_set):
      return 0


    def draw_som(self, imagepath):
      b_vector_label, b_target = get_labelAndTarget(self.training_set_features)
      dic_v2 = win_map(self.som, b_vector_label, return_indices=True)
      len(dic_v2.keys())
      newDic = get_labeledDic(self.som, b_vector_label, b_target)
      draw_labeled_neurons_map(self.som, newDic, imagepath,'png')
      return 0














In [ ]:
'''
from glob import glob

path_of_used_series_landmarks = []
path_of_used_series_images = []
for i in np.sort(glob("/content/SynthesisEmotions/ck+copyAgainT/Landmarks/*/*/", recursive = True)):

    subject = i.split('/')[5]
    serie = i.split('/')[6]
    actual = subject + '/'+ serie
    path = '/content/SynthesisEmotions/ck+copyAgainT/Emotion_labels/Emotion/' +actual + '/'
    image_path = '/content/SynthesisEmotions/ck+copyAgainT/extended-cohn-kanade-images/cohn-kanade-images/' +actual+ '/'
    landmark_path = '/content/SynthesisEmotions/ck+copyAgainT/Landmarks/' + actual+ '/'

    filelabel = np.sort((os.listdir(path)))[0]
    longitud = np.sort((os.listdir(str(i))))
    imagelabel = np.sort(os.listdir(str(image_path)))
    landmarklabel = np.sort(os.listdir(str(landmark_path)))
    with open(path+ '/'+filelabel) as f:
        line = float(f.readline())
    if line == 4 or line == 6:
        path_of_used_series_landmarks.append(landmark_path + landmarklabel[0])
        path_of_used_series_landmarks.append(landmark_path + landmarklabel[len(longitud)-2])
        path_of_used_series_landmarks.append(landmark_path + landmarklabel[len(longitud)-1])
        path_of_used_series_images.append(image_path + imagelabel[0])
        path_of_used_series_images.append(image_path + imagelabel[len(longitud)-2])
        path_of_used_series_images.append(image_path + imagelabel[len(longitud)-1])
    else:
        path_of_used_series_landmarks.append(landmark_path + landmarklabel[0])
        path_of_used_series_landmarks.append(landmark_path + landmarklabel[len(longitud)-1])
        path_of_used_series_images.append(image_path + imagelabel[0])
        path_of_used_series_images.append(image_path + imagelabel[len(longitud)-1])

from glob import glob

path_of_used_series_landmarks_test = []
path_of_used_series_images_test = []
for i in np.sort(glob("/content/SynthesisEmotions/ck+copyAgain/Landmarks/*/*/", recursive = True)):
    subject = i.split('/')[5]
    serie = i.split('/')[6]
    actual = subject + '/'+ serie
    path = '/content/SynthesisEmotions/ck+copyAgain/Emotion_labels/Emotion/' +actual + '/'
    image_path = '/content/SynthesisEmotions/ck+copyAgain/extended-cohn-kanade-images/cohn-kanade-images/' +actual+ '/'
    landmark_path = '/content/SynthesisEmotions/ck+copyAgain/Landmarks/' + actual+ '/'
    if actual != 'S084/004':
      filelabel = np.sort((os.listdir(path)))[0]
      longitud = np.sort((os.listdir(str(i))))
      imagelabel = np.sort(os.listdir(str(image_path)))
      landmarklabel = np.sort(os.listdir(str(landmark_path)))
      #path_of_used_series_landmarks_test.append(landmark_path + landmarklabel[0])
      path_of_used_series_landmarks_test.append(landmark_path + landmarklabel[len(longitud)-1])
      #path_of_used_series_images_test.append(image_path + imagelabel[0])
      path_of_used_series_images_test.append(image_path + imagelabel[len(longitud)-1])
#S084/04 la saqué porque era disgust

training_img_set =  path_of_used_series_images
training_lab_set = path_of_used_series_landmarks
testing_img_set = path_of_used_series_images_test
testing_lab_set = path_of_used_series_landmarks_test
alig_im= '/content/probando_aligned_img'
alig_lan = '/content/probando_aligned_landmarks.csv'
diff_path = '/content/probando_shapediff.csv'
alig_im_test= '/content/probando_aligned_img_test'
alig_lan_test = '/content/probando_aligned_landmarks_test.csv'
diff_path_test = '/content/probando_shapediff_test.csv'
structure_f_path = '/content/probando_struct_features.csv'
structure_f_path_test = '/content/probando_struct_features_test.csv'
cartoon_folder_path = '/content/probando_cartoon_folder'
texture_folder_path = '/content/probando_texture_folder'
cropped_img_path = '/content/cropped_img.png'
texture_save_path ='/content/texture_save.csv'
cartoon_save_path ='/content/cartoon_save.csv'
cartoon_folder_path_test = '/content/probando_cartoon_folder_test'
texture_folder_path_test = '/content/probando_texture_folder_test'
cropped_img_path_test = '/content/cropped_img_test.png'
texture_save_path_test ='/content/texture_save_test.csv'
cartoon_save_path_test ='/content/cartoon_save_test.csv'

classi = EmotionClassifier(training_img_set, training_lab_set, alig_im, alig_lan, diff_path)
'''


In [ ]:
#print(np.sort(np.asarray(testing_img_set)).shape)


In [ ]:
#test_shape = classi._get_testing_shape_feature(testing_img_set, testing_lab_set, alig_im_test, alig_lan_test, diff_path_test)

In [ ]:
#shape = classi._get_training_shape_feature()